In [2]:
!pip install requests beautifulsoup4 psycopg2-binary



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import psycopg2

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       # Ganti dengan nama database Anda
    user="postgres",         # Ganti dengan username PostgreSQL Anda
    password="felixexe131105",     # Ganti dengan password PostgreSQL Anda
    host="localhost",         # Jika PostgreSQL berjalan di komputer lokal
    port="5432"                   # Port default PostgreSQL
)

cursor = connection.cursor()
print("Koneksi berhasil!")


Koneksi berhasil!


In [19]:
import psycopg2
import requests
from bs4 import BeautifulSoup

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       # Ganti dengan nama database Anda
    user="postgres",             # Ganti dengan username PostgreSQL Anda
    password="felixexe131105",   # Ganti dengan password PostgreSQL Anda
    host="localhost",            # Ganti dengan host jika berbeda
    port="5432"                  # Port default PostgreSQL
)

cursor = connection.cursor()

# URL dari halaman e-commerce (contoh halaman produk)
url = "https://www.tokopedia.com/search?st=&q=Sabun&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="

# Lakukan HTTP GET request
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

# Parse konten halaman menggunakan BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Cari elemen HTML yang berisi nama dan harga produk
produk = []
for item in soup.find_all("div", class_="_6+OpBPVGAgqnmycna+bWIw=="):  # Sesuaikan selector berdasarkan situs
    nama = item.find("span", class_="_0T8-iGxMpV6NEsYEhwkqEg==").get_text()  # Selector untuk nama produk
    
    # Cari harga produk
    harga_item = item.find_next("div", class_="_67d6E1xDKIzw+i2D2L0tjw== t4jWW3NandT5hvCFAiotYg==")
    if harga_item:
        harga = harga_item.get_text().strip()  # Ambil harga dan hilangkan spasi
        harga = int(harga.replace("Rp", "").replace(".", ""))  # Konversi harga ke integer (hapus "Rp" dan titik)
        produk.append((nama, harga))

# Masukkan data ke tabel ProdukEcommerce
for p in produk:
    try:
        # Query untuk memasukkan data ke dalam tabel
        query = "INSERT INTO ProdukEcommerce (nama_produk, harga) VALUES (%s, %s)"
        cursor.execute(query, (p[0], p[1]))  # Eksekusi query dengan data nama dan harga produk
        print(f"Data berhasil dimasukkan: {p[0]} | Harga: {p[1]}")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")

# Commit perubahan ke database
connection.commit()

# Tutup koneksi
cursor.close()
connection.close()


Data berhasil dimasukkan: Dettol Sabun Mandi Cair Anti Bakteri Cool Pouch 800ml | Harga: 38070
Data berhasil dimasukkan: Dove Bar Soap - Go Fresh Fresh Moisture 3 x 100g | Harga: 121700
Data berhasil dimasukkan: [Buy 2 Get 2] Lifebuoy Sabun Cair dan Mandi Antiseptik Total 10 825ml | Harga: 121700
Data berhasil dimasukkan: Dettol Sabun Mandi Cair Activ-Botany Green Tea Bergamot Pouch 370gr | Harga: 24600
Data berhasil dimasukkan: Sabun Bee and Flower | Harga: 19500
Data berhasil dimasukkan: Lifebuoy Sabun Batang Antiseptik Total 10 110G - Bundle Isi 4 | Harga: 19500
Data berhasil dimasukkan: Nuvo Family Sabun Mandi Cair Total Protect Pouch 825Ml | Harga: 71550
Data berhasil dimasukkan: Giv White Sabun Mandi Mulberry Pouch 825 ml | Harga: 71550
Data berhasil dimasukkan: Biore Pure Mild 800ml Pouch Triplepack - Sabun Mandi Cair | Harga: 71550
